## Scraping Detik

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [3]:
jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'ganjar'

In [4]:
def scrape_links(page_number,keywords):
    url = f"https://www.detik.com/search/searchall?query={keywords}&sortby=time&page={page_number}"
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [5]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


https://www.detik.com/search/searchall?query=ganjar&sortby=time&page=1
Scraped 9 links from page 1
Total Links: 9


In [6]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "detail__title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "detail__author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "detail__date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('div',{"class": "page__breadcrumb"})
            if category_elements:
                category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail__body"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [7]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [8]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 9


,title,keywords,author,category,date,content,link
0,"Diajak Puan Kerja Sama Dukung Ganjar, Kaesang:...",ganjar,Annisa A,Pemilu,"Selasa, 26 Sep 2023 22:37 WIB",Ketum PSI Kaesang Pangarep menjawab ajakan Ket...,https://news.detik.com/pemilu/d-6952320/diajak...
1,Jokowi soal Projo Dukung Capres Inisial P: Kok...,ganjar,Tim detikcom,Pemilu,"Selasa, 26 Sep 2023 21:41 WIB",Ketua Umum Relawan Projo Budi Arie mengungkap ...,https://news.detik.com/pemilu/d-6952245/jokowi...
2,Puan Anggap Tak Ada Manuver Keluarga Jokowi Di...,ganjar,Adrial Akbar,Pemilu,"Selasa, 26 Sep 2023 20:20 WIB",Ketua DPP PDI Perjuangan (PDIP) Puan Maharani ...,https://news.detik.com/pemilu/d-6952137/puan-a...
3,"Kaesang Jadi Ketum PSI, Puan Maharani: Ikut PD...",ganjar,Author not found,Category not found,"4,552 Views | Selasa, 26 Sep 2023 20:21 WIB","Ketua DPP PDIP, Puan Maharani, memberikan sela...",https://news.detik.com/detiktv/d-6952238/kaesa...
4,"Bukan Anies, Ini Capres yang Akan Didukung Projo",ganjar,Tim detikJateng,Berita,"Rabu, 27 Sep 2023 01:00 WIB",Bacapres Koalisi Perubahan untuk Persatuan (KP...,https://www.detik.com/sumut/berita/d-6952253/b...
5,Kaesang Jawab Ajakan Puan Kerja Sama Dukung Ga...,ganjar,Tim detikNews,Berita,"Selasa, 26 Sep 2023 23:40 WIB",Ketua DPP PDIP Puan Maharani mengajak Kaesang ...,https://www.detik.com/jateng/berita/d-6952335/...
6,"Ganjar Dapat Jersey MU Nomor 8 dari PAPDESI, S...",ganjar,Dea Duta Aulia,Berita,"Selasa, 26 Sep 2023 21:05 WIB",Ketua Umum DPP Perkumpulan Aparatur Pemerintah...,https://www.detik.com/jateng/berita/d-6952213/...
7,Puan Bocorkan Obrolan Mega-Prabowo Saat Duduk ...,ganjar,Tim detikNews,Berita,"Selasa, 26 Sep 2023 21:13 WIB","Ketua DPP PDIP, Puan Maharani, mengungkap obro...",https://www.detik.com/jateng/berita/d-6952224/...
8,"Beri Selamat ke Kaesang Jadi Ketum PSI, Puan ...",ganjar,Tim detikNews,Berita,"Rabu, 27 Sep 2023 02:00 WIB",Ketua DPP PDI Perjuangan (PDIP) Puan Maharani ...,https://www.detik.com/sumut/berita/d-6951907/b...


Save Data

In [9]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/detik_{keywords}_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')